### Example usage for calculate_metrics

In [8]:
import pickle
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils.audio_to_spectrograms import create_spectrogram_pkl
#from src.main import calculate_metrics

# Copied from src.main as I'm having problems with audio mamba imports
from src.config import CLASSES
from loguru import logger
import sed_eval

def calculate_metrics(pred_event_dict, gt_event_dict, time_resolution=1.0, t_collar=0.25):
    logger.warning("Metrics not implemented!")
    # Based on event dictionary

    # Accuracy

    # IoU

    # DCASE SED eval: https://tut-arg.github.io/sed_eval/tutorial.html#id1
    event_based_metrics = sed_eval.sound_event.EventBasedMetrics(CLASSES, t_collar=t_collar)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        event_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(event_based_metrics)

    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(CLASSES, time_resolution=time_resolution)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        segment_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(segment_based_metrics)

In [9]:
# refer to single_stage_yamnet_frame.ipynb (Section "Get test set event predictions") for pickle file generation
pred_event_dict = pickle.load(open('outputs/single_stage_yamnet.pkl', 'rb'))

In [10]:
pred_event_dict['test_snipped_scene_0000.wav']

[{'file': 'test_snipped_scene_0000.wav',
  'event_onset': 0.96,
  'event_offset': 3.84,
  'event_label': 'siren'},
 {'file': 'test_snipped_scene_0000.wav',
  'event_onset': 3.84,
  'event_offset': 5.76,
  'event_label': 'car_horn'}]

In [11]:
# check if pkl file is created
gt_pkl_path = 'data/processed/yamnet/spectrograms_test_list.pkl'
if not(os.path.exists(gt_pkl_path)):
    create_spectrogram_pkl()

In [12]:
gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':ref_event['event_label']}]
                    for ref_event in gt_events}

In [13]:
gt_event_dict['test_snipped_scene_0002.wav']

[{'file': 'test_snipped_scene_0002.wav',
  'event_onset': np.float64(3.59310837808072),
  'event_offset': np.float64(6.195754211414053),
  'event_label': 'cough'}]

In [14]:
calculate_metrics(pred_event_dict, gt_event_dict, t_collar=0.5)

2025-12-02 20:33:09.615 | WARNING  | __main__:calculate_metrics:15 - Metrics not implemented!


Event based metrics (onset-offset)
  Evaluated length                  : 4111.27 sec
  Evaluated files                   : 500 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 500.00 ms
  Offset (length)                   : 50.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 48.76 %
    Precision                       : 45.14 %
    Recall                          : 53.00 %
  Error rate
    Error rate (ER)                 : 1.09 
    Substitution rate               : 0.02 
    Deletion rate                   : 0.45 
    Insertion rate                  : 0.62 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 49.46 %
    Precision                       : 46.35 %
    Recall                          : 53.20 %
  Error rate
    Error rate (ER)                 : 1.11 
    Deletion rate                   : 0.47 
    Insertion rate 